In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import cv2
import pickle as pkl
import PIL
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
import time
from scipy.spatial.transform import Rotation as R

In [ ]:
from ibmmpy.src.ibmmpy.ibmm import EyeClassifier

In [ ]:
dataset_root = Path("/scratch/abhijatb/Bosch22/dreyevr_recordings_sensors/LBC_config_gaze/")
route_data_path = Path("/scratch/abhijatb/Bosch22/dreyevr_recordings_sensors/LBC_config_gaze/exp_brady32")

In [ ]:
# read focus hitpts from the aux replay outputs and parse them

In [ ]:
# match them up to the OG replay dataframes

In [ ]:
# then do the 3 sensor gaze overlay with corrected focus hitpts
# (do gaze event classification)
# then convert to attention maps (history + dispersion)